In [ ]:
###
#  FUTON Model MDP + Q-Learning Creation Script
#  A Research Project conducted by Noah Dunn 
###

# Import the standard tools for working with Pandas dataframe
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shelve
# Import the MDP toolbox that contains a method for conducting Q-Learning
# Tool can be found here: https://github.com/sawcordwell/pymdptoolbox
# Documentation for the tool can be found here 
import mdptoolbox

In [1]:
# Python has object serialization to make write/reads fasters, in the form of pickle
import pickle

# Read these values back in from being saved to file
cluster_values = [];
cluster_labels = []; 
sample_zscores = []

with open ('cluster_centers.txt', 'rb') as fp:
    cluster_values = pickle.load(fp)
with open ('cluster_labels.txt', 'rb') as fp:
    cluster_labels = pickle.load(fp)
with open ('sample_zscores.txt', 'rb') as fp:
    sample_zscores = pickle.load(fp)
    
print(cluster_values, "\n", cluster_labels, "\n", sample_zscores)

[[-0.2254902  -0.32352941 -0.42817647 ... -0.07048865  0.20045151
   0.04158715]
 [ 0.09405941 -0.04455446 -0.47672277 ...  0.66545596  0.32843116
   0.42501051]
 [ 0.04545455 -0.15454545 -0.49425455 ...  0.30664516  0.28904419
   0.42852166]
 ...
 [-0.34615385 -0.38461538 -0.46365385 ... -1.47820946 -0.18871191
   0.04611651]
 [ 0.01351351 -0.01351351 -0.46678378 ...  0.34582232  0.109389
   0.14752571]
 [ 0.5        -0.5        -0.5        ... -1.52106461 -2.31206609
  -1.85290276]] 
 [321 229 527 ... 137 573 155] 
         gender  mechvent  max_dose_vaso  re_admission  qSOFAFlag  SOFAFlag  \
2         -0.5       0.5           -0.5     -2.302585        0.5       0.5   
5         -0.5      -0.5           -0.5     -2.302585        0.5       0.5   
9         -0.5      -0.5           -0.5     -2.302585        0.5       0.5   
11        -0.5      -0.5           -0.5     -2.302585        0.5       0.5   
20         0.5      -0.5           -0.5      0.095310        0.5      -0.5   
...     

In [7]:
# We now want to use the clusters to determine their nearest real data point neighbors
# As a visual of this. Suppose we have 4 flags of different colors scattered over a park. The K-Means++ algorithm
# is what planted the flags in the middle of groups of people that are similar. The KNN Search (K nearest neighbor search)
# enables us to know which people are near which flags. In SciPy, this is done with the Nearest Centroid algorithm
from sklearn.neighbors import NearestCentroid
knn_neighbors_alg = NearestCentroid()
nearest_neighbors_indexes = knn_neighbors_alg.fit(sample_zscores, cluster_values)

ValueError: bad input shape (750, 50)

In [ ]:
# The initial MDP matrix
# We need the values of weights that determines how much the model
# prefers transitioning from one state (medical conditional), to another
# The Matrix must be in the form [[S1][S2][A]] Where S1 is initial state, S2 is the second state, and
# A is the action taken to get from S1 to S2. 
transitions = [[][][]]

# We need to determine the reward value for predicting an outcome leading to survival (+)
# and a penalty for an outcome that will yield death (-)
# The Matrix must be in the form [[S1][S2][R]] Where S1 is initial state, S2 is the second state, and
# R is the reward for taking the action from S1 to S2. 
reward = [[][][]]

# We need to determine the discount value to influence the model to continue changing
# when outcomes are not desired, This value should be kept in the range 0 < discount < 1
discount = 1

# The Q-Learning algorithm will run a fixed number of times
numOfIterations = 10000

# We need to determine whether or not we waant to validate that the transitions and rewards matrix
# to make sure they are valid, this option will only be turned off for speed
scheck = False
